#### Make sure your solr sever is on
#### Core of washington Post version 3 need to be indexed before

In [18]:
import sys,os,time,re,csv,string
import pysolr
import pandas as pd
import numpy as np
import math
import nltk
import torch
from tqdm import tqdm
from ast import literal_eval
from datetime import datetime
import nltk
import torch
from sklearn.utils import shuffle
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
stop = set(stopwords.words('english'))
sys.path.append('../Evaluation/scripts/')
sys.path.append('../Pipeline/')

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/jovyan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
from solrclient import *
from runeval import runeval2020
from runs_export import *

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/jovyan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [21]:
#read query file of 2020
query_path = '../Data/Topic/trec2020/topicquery.csv'
queries = pd.read_csv(query_path)
queries = queries.rename(columns={'num': 'qnum'})
queries.head(1)

,qnum,docid,url_x,title,author,url_y,published_date,type,source,content,Query,Key_Words
0,Number: 886,AEQZNZSVT5BGPPUTTJO7SNMOLE,https://www.washingtonpost.com/politics/2019/0...,Trump’s own Defense Department directly contra...,Aaron Blake,/politics/2019/06/05/trump-says-transgender-tr...,NaN,NaN,NaN,The FixTrump’s own Defense Department directly...,"['can transgender troops serve', 'can transgen...","['members', 'hormone deficiency', 'transgender..."


In [6]:
def retrieval(resftp,queries):   
    rows = 200
    for i in range(len(queries)):
        client = SolrClient2020(server_port='8983', server_core='WapoV3')
        query = queries.loc[i,:]
        querytext = query['title']        
        client.get_results(query,querytext,rows)
        client.export_results(resftp + query['qnum'][-3:]+'.csv')
    print("Retrieval successed!")

In [7]:
retrieval('./eval_result_2020/result/',queries)

Retrieval successed!


In [6]:
# evaluation
def evaluate(resfoldername,details=False,Exportmethod = ExportRun2020 ):
    resftp = './eval_result_2020/%s/'%(resfoldername)
    Exportmethod(resftp,resfoldername)
    #runpath = '/home/jovyan/playground/projects/TREC2021/NewsTrack/BaseLine/%s.csv.gz'%(resfoldername)
    if Exportmethod == ExportRun2020Temp:
        resfoldername += "temp"
    runpath = '/home/jovyan/playground/projects/TREC2021/NewTrack_Github/Classifier/eval_result_2020/'+ '%s.csv.gz'%(resfoldername)
    evalpath = '/home/jovyan/playground/projects/TREC2021/NewTrack_Github/Evaluation/res_topics/%s.trec_eval'%(resfoldername)
    if details:
        scores = runeval2020(runpath,evalpath,detail=True)
        return scores
    else:
        runeval2020(runpath,evalpath)

In [5]:
# evaluate("result")
scores = evaluate("result",details=True)
scores

Got 50 result files


../Pipeline/runs_export.py:73: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rundf['runtag'] = resname.replace(".csv","")


Total length is : 5000
Doing Evaluation ...
cd /home/jovyan/playground/projects/TREC2021/NewTrack_Github/Evaluation/scripts && python eval-background.py --qrels /home/jovyan/playground/projects/TREC2021/NewTrack_Github/Evaluation/eval/2020/qrels.background /home/jovyan/playground/projects/TREC2021/NewTrack_Github/Classifier/eval_result_2020/result.csv.gz /home/jovyan/playground/projects/TREC2021/NewTrack_Github/Evaluation/res_topics/result.trec_eval
trec.eval file has been save to /home/jovyan/playground/projects/TREC2021/NewTrack_Github/Evaluation/res_topics/result.trec_eval


[['ndcg_cut_5', '886', '0.4122'],
 ['ndcg_cut_5', '887', '0.6362'],
 ['ndcg_cut_5', '888', '0.0000'],
 ['ndcg_cut_5', '889', '0.4778'],
 ['ndcg_cut_5', '890', '0.4024'],
 ['ndcg_cut_5', '891', '0.2844'],
 ['ndcg_cut_5', '892', '0.3187'],
 ['ndcg_cut_5', '893', '0.2140'],
 ['ndcg_cut_5', '894', '0.3097'],
 ['ndcg_cut_5', '895', '0.2926'],
 ['ndcg_cut_5', '896', '0.3543'],
 ['ndcg_cut_5', '897', '1.0000'],
 ['ndcg_cut_5', '898', '0.3955'],
 ['ndcg_cut_5', '899', '0.7322'],
 ['ndcg_cut_5', '900', '0.3066'],
 ['ndcg_cut_5', '901', '0.0000'],
 ['ndcg_cut_5', '902', '0.0000'],
 ['ndcg_cut_5', '903', '0.0689'],
 ['ndcg_cut_5', '904', '0.0980'],
 ['ndcg_cut_5', '905', '0.0000'],
 ['ndcg_cut_5', '906', '0.3322'],
 ['ndcg_cut_5', '907', '0.0000'],
 ['ndcg_cut_5', '908', '0.0000'],
 ['ndcg_cut_5', '909', '0.6578'],
 ['ndcg_cut_5', '910', '0.1520'],
 ['ndcg_cut_5', '911', '0.8614'],
 ['ndcg_cut_5', '912', '0.0000'],
 ['ndcg_cut_5', '913', '0.2676'],
 ['ndcg_cut_5', '914', '0.3016'],
 ['ndcg_cut_5'

In [8]:
scorestmp = evaluate("result",details=True,Exportmethod = ExportRun2020Temp)
scorestmp

Got 50 result files
25 topics have no publish date.
Total length is : 5000
Doing Evaluation ...
cd /home/jovyan/playground/projects/TREC2021/NewTrack_Github/Evaluation/scripts && python eval-background.py --qrels /home/jovyan/playground/projects/TREC2021/NewTrack_Github/Evaluation/eval/2020/qrels.background /home/jovyan/playground/projects/TREC2021/NewTrack_Github/Classifier/eval_result_2020/resulttemp.csv.gz /home/jovyan/playground/projects/TREC2021/NewTrack_Github/Evaluation/res_topics/resulttemp.trec_eval
trec.eval file has been save to /home/jovyan/playground/projects/TREC2021/NewTrack_Github/Evaluation/res_topics/resulttemp.trec_eval


[['ndcg_cut_5', '886', '0.4122'],
 ['ndcg_cut_5', '887', '0.6362'],
 ['ndcg_cut_5', '888', '0.0000'],
 ['ndcg_cut_5', '889', '0.4492'],
 ['ndcg_cut_5', '890', '0.4024'],
 ['ndcg_cut_5', '891', '0.2299'],
 ['ndcg_cut_5', '892', '0.3187'],
 ['ndcg_cut_5', '893', '0.6844'],
 ['ndcg_cut_5', '894', '0.2719'],
 ['ndcg_cut_5', '895', '0.4741'],
 ['ndcg_cut_5', '896', '0.2712'],
 ['ndcg_cut_5', '897', '0.4704'],
 ['ndcg_cut_5', '898', '0.0000'],
 ['ndcg_cut_5', '899', '0.7167'],
 ['ndcg_cut_5', '900', '0.9197'],
 ['ndcg_cut_5', '901', '0.0000'],
 ['ndcg_cut_5', '902', '0.0000'],
 ['ndcg_cut_5', '903', '0.1924'],
 ['ndcg_cut_5', '904', '0.0980'],
 ['ndcg_cut_5', '905', '0.0000'],
 ['ndcg_cut_5', '906', '0.3761'],
 ['ndcg_cut_5', '907', '0.0000'],
 ['ndcg_cut_5', '908', '0.0183'],
 ['ndcg_cut_5', '909', '0.6578'],
 ['ndcg_cut_5', '910', '0.0760'],
 ['ndcg_cut_5', '911', '0.8614'],
 ['ndcg_cut_5', '912', '0.0000'],
 ['ndcg_cut_5', '913', '0.2676'],
 ['ndcg_cut_5', '914', '0.3016'],
 ['ndcg_cut_5'

In [14]:
def get_ongoingtopics(scores,scorestmp):
    atemporal = []
    temporal = []
    for i in range(len(scores)):
        if scores[i][2] < scorestmp[i][2]:
            if scorestmp[i][1] != 'all':
                temporal.append(scorestmp[i][1])
        elif scores[i][2] > scorestmp[i][2]:
            if scorestmp[i][1] != 'all':
                atemporal.append(scorestmp[i][1])
    return temporal, atemporal

In [15]:
ongoing_topics, other_topics = get_ongoingtopics(scores,scorestmp)

In [16]:
ongoing_topics

['893', '895', '900', '903', '906', '908', '916', '928', '933']

In [17]:
other_topics

['889',
 '891',
 '894',
 '896',
 '897',
 '898',
 '899',
 '910',
 '921',
 '922',
 '925',
 '927',
 '931']

In [27]:
def cons_traindata(query_path,ongoing_topics,other_topics):
    topics = pd.read_csv(query_path)
    rlistsen,alistsen = [],[]
    for i in range(len(topics)):
        if topics['num'][i][-3:] in ongoing_topics:
            rlistsen.append(topics['title'][i])
        elif topics['num'][i][-3:] in other_topics:
            alistsen.append(topics['title'][i])
            
    # load temporalia topics
    df_temporalia = pd.read_csv('./query.csv')
    atemporal = df_temporalia[df_temporalia['qtype']=='atemporal']
    atemporal['label']='0'
    atemporal = atemporal[["query",'label']]
    recency = df_temporalia[df_temporalia['qtype']=='recency']
    recency['label']="1"
    recency = recency[["query",'label']]
    
    
    # add topic2020
    for sen in rlistsen:
        new_row = {'query':sen, 'label':1}
        recency = recency.append(new_row, ignore_index=True)
    
    for sen in alistsen:
        new_row = {'query':sen, 'label':0}
        atemporal = atemporal.append(new_row, ignore_index=True)
        
    #create train,test,validate pairs
    total = atemporal.append(recency)
    total = shuffle(total)
    total.rename(columns={'query':'queries'},inplace=True)
    test = total.sample(12)
    train = total[~total.queries.isin(test.queries)]
    total.sample(12).to_csv('valid1.csv',index=False)
    test.to_csv('test1.csv',index=False)
    train.to_csv('train1.csv',index=False)
    print("train data created")

In [28]:
cons_traindata(query_path,ongoing_topics,other_topics)

train data created


<ipython-input-27-7e390042a022>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  atemporal['label']='0'
<ipython-input-27-7e390042a022>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  recency['label']="1"
